## Algoritmo de optimización



In [22]:
 # -*- coding: utf-8 -*- 
import json
import uuid
import datetime
from collections import OrderedDict, defaultdict
import numpy as np
from scipy.optimize import minimize 
import logging 
import os 

# Configuración básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- Parámetros ---
# Ponderaciones de la función objetivo
alpha1, alpha2 = 1.0, 100.0 # Mantener penalización fuerte para coordinación

# Parámetros curva IEC Standard Inverse
k, n   = 0.14, 0.02 

# Intervalo de Tiempo de Coordinación (CTI)
CTI = 0.2 

# Límites para pick_up (Ipi) y TDS
IPI_MIN, IPI_MAX = 0.1, 5.0   
TDS_MIN, TDS_MAX = 0.05, 1.1 
TDS_STEP = 0.01 

# Rutas de entrada/salida (Asegúrate que sean correctas)
INPUT_JSON  = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs.json" 
# Nombre de salida para esta versión
OUTPUT_JSON = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/optimized_relay_values_paper_slsqp_relaxed.json" 

# --- Funciones (timeout, objective_scenario, constraint_delta_t - sin cambios) ---

def timeout(Ishc, Ipi, TDS):
    """Cálculo de timeout bajo curva inversa estándar."""
    if Ipi <= 0: return 1000.0
    ratio = Ishc / Ipi
    if ratio <= 1.0 or TDS <= 0: return 1000.0 
    try: power_term = np.power(ratio, n) 
    except OverflowError: return 1000.0
    if power_term <= 1.0: return 1000.0
    denominator = power_term - 1
    if denominator < 1e-9: return 1000.0 
    time = (k / denominator) * TDS
    MAX_TIMEOUT = 100.0 
    if time < 0 or time > MAX_TIMEOUT or not np.isfinite(time): return MAX_TIMEOUT
    return time

def objective_scenario(x, pairs, relay_idx, alpha1_param, alpha2_param, k_param, n_param, cti_param):
    """Función objetivo: Minimiza tiempos y penaliza descoordinación respecto a CTI."""
    obj_func_val = 0.0; term1_sum = 0.0; term2_sum = 0.0
    calculated_timeouts = {} 
    for p_idx, p in enumerate(pairs): 
        main_relay_name = str(p["main_relay"]["relay"]); backup_relay_name = str(p["backup_relay"]["relay"]) 
        if main_relay_name not in relay_idx or backup_relay_name not in relay_idx: logger.error(f"[Obj] Relé no hallado P{p_idx} M:{main_relay_name} B:{backup_relay_name}"); continue 
        im = relay_idx[main_relay_name]; ib = relay_idx[backup_relay_name]
        if 2*im >= len(x) or 2*ib >= len(x): logger.error(f"[Obj] Índice fuera rango im={im}, ib={ib}, len(x)={len(x)}"); continue
        pu_m, tds_m = x[2*im], x[2*im+1]; pu_b, tds_b = x[2*ib], x[2*ib+1]
        try: mI = float(p["main_relay"]["Ishc"]); bI = float(p["backup_relay"]["Ishc"]) 
        except (ValueError, TypeError, KeyError) as e: logger.error(f"[Obj] Error Ishc P{p_idx} ({main_relay_name}/{backup_relay_name}): {e}"); continue
        key_m = (im, mI); key_b = (ib, bI) 
        if key_m not in calculated_timeouts: calculated_timeouts[key_m] = timeout(mI, pu_m, tds_m)
        t_m = calculated_timeouts[key_m]
        if key_b not in calculated_timeouts: calculated_timeouts[key_b] = timeout(bI, pu_b, tds_b)
        t_b = calculated_timeouts[key_b]
        if not np.isfinite(t_m) or not np.isfinite(t_b): term1 = 1e10; term2 = 1e10; logger.warning(f"[Obj] Tiempo no finito P{p_idx} ({main_relay_name}/{backup_relay_name}) t_m={t_m}, t_b={t_b}.")
        else: term1 = alpha1_param * (t_m**2 + t_b**2); delta_t = t_b - t_m; miscoordination_penalty = max(0, cti_param - delta_t); term2 = alpha2_param * (miscoordination_penalty**2)
        term1_sum += term1; term2_sum += term2; obj_func_val += term1 + term2
    if not np.isfinite(obj_func_val): logger.warning(f"[Obj] OF no finito ({obj_func_val})"); return 1e20 
    return obj_func_val

def constraint_delta_t(x, p, relay_idx, k_param, n_param, cti_param):
    """Constraint function para SLSQP: delta_t = t_b - t_m >= CTI."""
    main_relay_name = str(p["main_relay"]["relay"])
    backup_relay_name = str(p["backup_relay"]["relay"])
    if main_relay_name not in relay_idx or backup_relay_name not in relay_idx: logger.error(f"[Constr] Relé no hallado M:{main_relay_name} B:{backup_relay_name}"); return -1e6 
    im = relay_idx[main_relay_name]; ib = relay_idx[backup_relay_name]
    if 2*im >= len(x) or 2*ib >= len(x): logger.error(f"[Constr] Índice fuera rango im={im}, ib={ib}, len(x)={len(x)}"); return -1e6
    pu_m, tds_m = x[2*im], x[2*im+1]; pu_b, tds_b = x[2*ib], x[2*ib+1]
    try: mI = float(p["main_relay"]["Ishc"]); bI = float(p["backup_relay"]["Ishc"])
    except (ValueError, TypeError, KeyError): logger.error(f"[Constr] Error Ishc {main_relay_name}/{backup_relay_name}"); return -1e6 
    t_m = timeout(mI, pu_m, tds_m); t_b = timeout(bI, pu_b, tds_b)
    if not np.isfinite(t_m) or not np.isfinite(t_b): logger.warning(f"[Constr] Tiempo no finito {main_relay_name}/{backup_relay_name}. t_m={t_m}, t_b={t_b}"); return -1e6 
    constraint_value = t_b - t_m - cti_param
    return constraint_value

# --- Función Principal ---

def run_and_unify(input_json_path, output_json_path, cti_value):
    # ... (Carga y validación de datos igual que antes) ...
    try:
        with open(input_json_path, "r", encoding='utf-8') as f: data = json.load(f)
        logger.info(f"Datos cargados desde: {input_json_path}")
    except FileNotFoundError: logger.error(f"Archivo no encontrado: {input_json_path}"); return
    except json.JSONDecodeError as e: logger.error(f"JSON inválido: {input_json_path}: {e}"); return
    except Exception as e: logger.error(f"Error lectura archivo {input_json_path}: {e}"); return
    by_scenario = defaultdict(list); required_keys = ["scenario_id", "main_relay", "backup_relay"]; required_relay_keys = ["relay", "Ishc"] 
    valid_pairs_count = 0; skipped_pairs_count = 0
    if not isinstance(data, list): logger.error(f"Contenido de {input_json_path} no es lista."); return
    for i, pair in enumerate(data): # Validación
        if not isinstance(pair, dict): logger.warning(f"Entrada {i} omitida: No es diccionario."); skipped_pairs_count += 1; continue
        if not all(k in pair for k in required_keys): logger.warning(f"Par {i} (ID: {pair.get('scenario_id', 'N/A')}) omitido: Faltan claves. Claves: {list(pair.keys())}"); skipped_pairs_count += 1; continue
        main_relay = pair.get("main_relay"); backup_relay = pair.get("backup_relay"); scenario_id_log = pair.get('scenario_id', 'N/A') 
        if not isinstance(main_relay, dict) or not isinstance(backup_relay, dict): logger.warning(f"Par {i} (Esc {scenario_id_log}) omitido: main/backup no son dict."); skipped_pairs_count += 1; continue
        if not all(k in main_relay for k in required_relay_keys) or not all(k in backup_relay for k in required_relay_keys): logger.warning(f"Par {i} (Esc {scenario_id_log}) omitido: Falta relay/Ishc. MainK: {list(main_relay.keys())}, BackupK: {list(backup_relay.keys())}"); skipped_pairs_count += 1; continue
        try: mI = float(main_relay['Ishc']); bI = float(backup_relay['Ishc']); assert np.isfinite(mI) and np.isfinite(bI) and mI > 0 and bI > 0
        except: logger.warning(f"Par {i} (Esc {scenario_id_log}) omitido: Ishc inválido (M='{main_relay.get('Ishc')}', B='{backup_relay.get('Ishc')}')."); skipped_pairs_count += 1; continue
        main_relay_name_val = main_relay.get("relay"); backup_relay_name_val = backup_relay.get("relay")
        if not isinstance(main_relay_name_val, (str, int, float)) or not isinstance(backup_relay_name_val, (str, int, float)) or str(main_relay_name_val).strip() == "" or str(backup_relay_name_val).strip() == "": logger.warning(f"Par {i} (Esc {scenario_id_log}) omitido: Nombre relé inválido (M='{main_relay_name_val}', B='{backup_relay_name_val}')."); skipped_pairs_count += 1; continue
        by_scenario[str(pair["scenario_id"])].append(pair); valid_pairs_count += 1
    logger.info(f"Agrupados {valid_pairs_count} pares válidos en {len(by_scenario)} escenarios. Omitidos: {skipped_pairs_count}.")
    if not by_scenario: logger.error("No hay escenarios válidos."); return
        
    unified = []
    optimization_stats = {}
    
    for scen, pairs in by_scenario.items(): 
        logger.info(f"--- Procesando Escenario: {scen} ({len(pairs)} pares) ---")
        if not pairs: logger.warning(f"Escenario {scen} sin pares válidos iniciales. Omitiendo."); scenario_result_data = { "_id": {"$oid": str(uuid.uuid4())}, "scenario_id": scen, "timestamp": datetime.datetime.utcnow().isoformat(timespec='milliseconds') + "Z", "optimization_method": None, "relay_values": {}, "optimization_status": -3, "optimization_success": False, "optimization_message": "Skipped - No valid pairs in input", "objective_function_value": None, "optimization_iterations": None, "optimization_time_seconds": 0.0 }; unified.append(scenario_result_data); optimization_stats[scen] = {"success": False, "status": -3, "message": "Skipped - No input pairs"}; continue 
        relays = OrderedDict(); 
        for p in pairs: relays[str(p["main_relay"]["relay"])] = None; relays[str(p["backup_relay"]["relay"])] = None
        relay_list = list(relays.keys()); relay_idx = {r: i for i, r in enumerate(relay_list)}; num_relays = len(relay_list); num_vars = num_relays * 2 
        logger.info(f"Relés únicos a optimizar en {scen}: {num_relays}")

        # Inicialización x0
        x0 = np.zeros(num_vars); initialized_relays = set()
        for p in pairs: 
             for side in ("main_relay", "backup_relay"):
                 relay_info = p[side]; r_name = str(relay_info["relay"])
                 if r_name not in initialized_relays and r_name in relay_idx: 
                     i = relay_idx[r_name]; pu_init_val = relay_info.get("pickup"); tds_init_val = relay_info.get("TDS")
                     try: pu_init = float(pu_init_val) if pu_init_val is not None else (IPI_MIN + IPI_MAX) / 2
                     except: pu_init = (IPI_MIN + IPI_MAX) / 2
                     try: tds_init = float(tds_init_val) if tds_init_val is not None else (TDS_MIN + TDS_MAX) / 2
                     except: tds_init = (TDS_MIN + TDS_MAX) / 2
                     x0[2*i] = np.clip(pu_init, IPI_MIN, IPI_MAX); x0[2*i+1] = np.clip(tds_init, TDS_MIN, TDS_MAX); initialized_relays.add(r_name)
                     
        # Límites (Bounds) para SLSQP (lista de tuplas)
        bnds = [(IPI_MIN, IPI_MAX), (TDS_MIN, TDS_MAX)] * num_relays 
        
        # Restricciones (Constraints) para SLSQP (lista de diccionarios)
        cons = []
        for p_idx, p in enumerate(pairs): 
             # Usar la función de restricción original (no vectorizada)
             constraint_func = lambda x, p_local=p: constraint_delta_t(x, p_local, relay_idx, k, n, cti_value)
             cons.append({"type": "ineq", "fun": constraint_func}) # 'ineq' significa func >= 0
        logger.info(f"Número de restricciones de coordinación para {scen}: {len(cons)}")

        # --- Ejecutar la optimización con SLSQP y tolerancias relajadas ---
        obj_args = (pairs, relay_idx, alpha1, alpha2, k, n, cti_value) 
        optimization_method = "SLSQP" # Volver a SLSQP
        logger.info(f"Iniciando optimización para {scen} con método {optimization_method} (tolerancias relajadas)...")
        
        # --- CAMBIO: Opciones con tolerancias más relajadas ---
        # ftol: Tolerancia en cambio de función objetivo
        # Aumentar ligeramente de 1e-7 a 1e-5 (o 1e-4 si sigue fallando)
        # Reducir maxiter un poco para evitar esperas largas si falla
        options = {'maxiter': 800, 'ftol': 1e-5, 'disp': False} 
                     
        start_time = datetime.datetime.now()
        try:
            if len(x0) != num_vars: raise ValueError(f"Tamaño x0 ({len(x0)}) != num_vars ({num_vars}) en {scen}")
            
            # Llamada a minimize con SLSQP y nuevas opciones
            res = minimize(
                objective_scenario, 
                x0, 
                args=obj_args, 
                method=optimization_method, 
                bounds=bnds, # Lista de tuplas para SLSQP
                constraints=cons, # Lista de diccionarios para SLSQP
                options=options
            )
            
            end_time = datetime.datetime.now(); duration = (end_time - start_time).total_seconds()
            logger.info(f"Optimización {scen} ({duration:.2f}s). Estado: {res.status} ({res.message})")
            if not res.success: logger.warning(f"Fallo optimización {scen}: {res.message} (Status={res.status})") # Log más claro del fallo
            # logger.debug(f"Resultado completo {scen}: {res}") # Descomentar para depuración extrema
            logger.info(f"Valor OF final {scen}: {res.fun if res.success else 'N/A'}") # Mostrar OF solo si tuvo éxito
            optimization_successful = res.success
                 
        except ValueError as e:
             end_time = datetime.datetime.now(); duration = (end_time - start_time).total_seconds()
             logger.error(f"Error (ValueError) optimización {scen} ({duration:.2f}s): {e}", exc_info=True)
             optimization_successful = False; res = None 
        except Exception as e:
             end_time = datetime.datetime.now(); duration = (end_time - start_time).total_seconds()
             logger.error(f"Excepción inesperada optimización {scen} ({duration:.2f}s): {e}", exc_info=True)
             optimization_successful = False; res = None

        # --- Procesar y guardar resultados ---
        # Formato de salida ajustado para coincidir con el ejemplo deseado
        
        scenario_result_data = {
             # "_id": {"$oid": str(uuid.uuid4())}, # Omitido si no es necesario para MongoDB
             "scenario_id": scen,
             "timestamp": datetime.datetime.utcnow().isoformat(timespec='microseconds') + "+00:00", # Formato ejemplo
             # Incluir detalles de optimización aunque falle, si están disponibles
             "optimization_details": { 
                 "method": optimization_method,
                 "success": False, # Default a Falso
                 "status": -1 if res is None else res.status,
                 "message": "Optimization failed due to error" if res is None else res.message,
                 "iterations": None,
                 "time_seconds": round(duration, 3),
                 "final_objective_value": None
             },
             "relay_values": {} # Default a vacío
        }

        if res is not None: # Actualizar detalles si res existe
             scenario_result_data["optimization_details"]["success"] = bool(res.success)
             scenario_result_data["optimization_details"]["status"] = res.status
             scenario_result_data["optimization_details"]["message"] = res.message
             if hasattr(res, 'nit'): scenario_result_data["optimization_details"]["iterations"] = int(res.nit)
             if res.success and np.isfinite(res.fun): scenario_result_data["optimization_details"]["final_objective_value"] = float(res.fun)
                 
        if optimization_successful and res is not None:
            x_opt = res.x.copy(); final_relay_values = {}
            for r_name, i in relay_idx.items(): 
                # Redondear TDS al paso más cercano ANTES de aplicar clip
                tds_opt_rounded = np.round(x_opt[2*i+1] / TDS_STEP) * TDS_STEP
                # Aplicar clip a ambos valores
                pickup_opt = np.clip(x_opt[2*i], IPI_MIN, IPI_MAX)
                tds_opt_final = np.clip(tds_opt_rounded, TDS_MIN, TDS_MAX)
                # Guardar con el formato del ejemplo
                final_relay_values[r_name] = {"TDS": float(tds_opt_final), "pickup": float(pickup_opt)} 
            
            scenario_result_data["relay_values"] = final_relay_values # Actualizar relay_values
            optimization_stats[scen] = {"success": True, "status": res.status, "message": res.message, "obj_func": res.fun}
        else:
            logger.warning(f"Optimización fallida/incompleta para {scen}. No se guardarán relay_values.")
            optimization_stats[scen] = {"success": False, "status": -1 if res is None else res.status, "message": "Failed/Error" if res is None else res.message}
        
        unified.append(scenario_result_data) # Añadir resultado (con o sin relay_values)

    # --- Fin del bucle de escenarios ---
    # ... (Resumen y guardado igual que antes) ...
    successful_scenarios = sum(1 for s in optimization_stats.values() if s["success"])
    skipped_scenarios = sum(1 for s in optimization_stats.values() if s.get("status") == -3) # Usar get por si acaso
    failed_scenarios = len(by_scenario) - successful_scenarios - skipped_scenarios
    logger.info(f"\n--- Resumen Optimización ---")
    logger.info(f"Escenarios leídos: {len(by_scenario)}")
    logger.info(f"  Optimizados Exitosamente: {successful_scenarios}")
    logger.info(f"  Fallidos/Incompletos: {failed_scenarios}")
    logger.info(f"  Omitidos (sin pares válidos iniciales): {skipped_scenarios}")
    if failed_scenarios > 0 or skipped_scenarios > 0:
        logger.warning("Escenarios con optimización fallida, incompleta u omitida:")
        for scen, stats in optimization_stats.items():
             if not stats.get("success", False): # Usar get por si acaso
                 status_desc = "Omitido (Sin Pares Iniciales)" if stats.get('status') == -3 else f"Fallido/Incompleto (Status={stats.get('status', 'N/A')})"
                 logger.warning(f"  - {scen}: {status_desc}, Message='{stats.get('message', 'N/A')}'")
    try:
        output_dir = os.path.dirname(output_json_path);
        if output_dir and not os.path.exists(output_dir): os.makedirs(output_dir); logger.info(f"Directorio salida creado: {output_dir}")
        with open(output_json_path, "w", encoding='utf-8') as f: json.dump(unified, f, indent=2, ensure_ascii=False) 
        logger.info(f"Archivo unificado guardado en: {output_json_path}")
    except IOError as e: logger.error(f"Error E/S al guardar JSON en {output_json_path}: {e}", exc_info=True)
    except Exception as e: logger.error(f"Error inesperado al guardar JSON: {e}", exc_info=True)

# --- Punto de Entrada ---
if __name__ == "__main__":
    run_and_unify(input_json_path=INPUT_JSON, 
                  output_json_path=OUTPUT_JSON, 
                  cti_value=CTI)

2025-04-20 14:11:12,946 - INFO - Datos cargados desde: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs.json
2025-04-20 14:11:12,950 - WARNING - Par 126 (Esc scenario_2) omitido: Ishc inválido (M='0.64', B='0').
2025-04-20 14:11:12,950 - WARNING - Par 129 (Esc scenario_2) omitido: Ishc inválido (M='0.43', B='0').
2025-04-20 14:11:12,951 - WARNING - Par 226 (Esc scenario_3) omitido: Ishc inválido (M='0.67', B='0').
2025-04-20 14:11:12,952 - WARNING - Par 229 (Esc scenario_3) omitido: Ishc inválido (M='0.45', B='0').
2025-04-20 14:11:12,952 - WARNING - Par 291 (Esc scenario_3) omitido: Ishc inválido (M='1.49', B='0').
2025-04-20 14:11:12,954 - WARNING - Par 426 (Esc scenario_5) omitido: Ishc inválido (M='0.65', B='0').
2025-04-20 14:11:12,955 - WARNING - Par 429 (Esc scenario_5) omitido: Ishc inválido (M='0.44', B='0').
2025-04-20 14:11:12,958 - WARNING - Par 526 (Esc scenario_6) omitido: Ishc inválido (M='0.65', B='0').
2025-04-20 14

KeyboardInterrupt: 